<a href="https://colab.research.google.com/github/southaisdev/projeto_controle_DM1/blob/main/Projeto_Controle_DM1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [17]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from datetime import date
from google.adk.tools import google_search
import textwrap
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings("ignore")

In [18]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [19]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
#####################################################
# --- Agente 1: Monitoramento e Análise de Dados --- #
#####################################################
def agente_monitoramento_dm1(dados_dia):
    monitorador = Agent(
        name="agente_monitoramento_dm1",
        model="gemini-2.0-flash",
        description="Agente que analisa dados de controle de DM1 e gera um resumo.",
        instruction=f"""
        Você é um assistente de monitoramento de dados de DM1. Sua tarefa é analisar os dados fornecidos para um período
        e gerar um resumo em linguagem natural, destacando a média, min, max, hipo/hiper, e padrões simples. Você pode usar a
        ferramenta de busca do Google (google_search) para mostrar de forma sucinta algumas informações relevantes se necessário.
        **NUNCA** forneça conselhos médicos, diagnósticos ou recomendações de tratamento.
        Use a data de hoje ({date.today().strftime("%d/%m/%Y")}) como referência.
        Formato dos dados de entrada esperado: Data/Hora, Glicemia (mg/dL), Refeição (Descrição e Carbos aprox), Insulina (Tipo e Unidades),
        Atividade (Descrição e Duração)
        """,
        tools=[google_search]
        )
    entrada_agente = f"Analise os seguintes dados:\n\n{dados_dia}"
    resumo_analise = call_agent(monitorador, entrada_agente)
    return resumo_analise


In [21]:
###############################################
# --- Agente 2: Conselheiro Nutricional --- #
###############################################
def agente_nutricional_dm1(refeicao_planejada, glicemia_atual_contexto=None):
    conselheiro = Agent(
        name="agente_nutricional_dm1",
        model="gemini-2.0-flash",
        description="Agente que oferece conselhos nutricionais gerais e informações sobre alimentos para pessoas com DM1.",
        instruction="""
        Você é um conselheiro nutricional amigável e educativo para pessoas com DM1.
        Sua função é fornecer informações sobre como diferentes tipos de alimentos podem afetar a glicemia e sugerir opções saudáveis (Geral).
        Pode fornecer estimativas *aproximadas* de carboidratos (sempre ressalte que são estimativas).
        Enfatize a importância de um plano alimentar individualizado e da consulta a um nutricionista ou médico.
        **NUNCA** diga o que *deve* comer ou relacione a refeição a uma dose de insulina específica.
        Considere a glicemia atual fornecida (se houver) para contextualizar a resposta.
        Você pode usar a ferramenta de busca do Google (google_search) para mostrar de forma sucinta algumas informações relevantes se necessário.
        """,
        tools=[google_search]
    )

    entrada_agente = f"Refeição ou alimento a ser considerado: {refeicao_planejada}"
    if glicemia_atual_contexto is not None:
         entrada_agente += f"\nGlicemia atual para contexto: {glicemia_atual_contexto} mg/dL"

    conselho_nutricional = call_agent(conselheiro, entrada_agente)
    return conselho_nutricional


In [22]:
#####################################################
# --- Agente 3: Suporte à Dosagem de Insulina --- #
#####################################################
def agente_insulina_dm1(glicemia_atual, carbos_refeicao, insulina_ativa=None, atividade_recente=None):
    suporte_dosagem = Agent(
        name="agente_insulina_dm1",
        model="gemini-2.0-flash",
        description="Agente que explica os fatores a serem considerados ao decidir a dose de insulina para DM1 (EDUCACIONAL).",
        instruction="""
        ATENÇÃO: Você é um assistente EDUCATIVO sobre os Fatores que influenciam a dosagem de insulina para DM1.
        SUA FUNÇÃO É EXPLICAR ESSES FATORES, **NUNCA CALCULAR OU RECOMENDAR UMA DOSE ESPECÍFICA DE INSULINA.**
        Explique como a glicemia atual, carboidratos, insulina ativa e atividade física *podem* influenciar a necessidade de insulina rápida.
        Sua tarefa é listar e explicar a relevância de cada fator.
        Sempre termine reforçando a importância de seguir as orientações do médico/equipe de saúde e que esta informação é apenas educativa.
        """,
        tools=[]
    )
    entrada_agente = f"Explique os fatores a considerar para a insulina rápida:\nGlicemia atual: {glicemia_atual} mg/dL\nCarboidratos planejados para a refeição: {carbos_refeicao} g"
    if insulina_ativa and insulina_ativa.lower() != 'nenhuma':
        entrada_agente += f"\nInsulina ativa ou última dose: {insulina_ativa}"
    if atividade_recente and atividade_recente.lower() != 'nenhuma':
        entrada_agente += f"\nAtividade física recente/planejada: {atividade_recente}"

    suporte_informativo = call_agent(suporte_dosagem, entrada_agente)
    return suporte_informativo

In [23]:
if __name__ == "__main__":
    print("🔬 Iniciando o Projeto Simplificado de Suporte a DM1 com Agentes 🔬")
    print("--- Por favor, insira os dados do seu registro ---")

    horario_input = input("⌚ Horário do registro (HH:MM): ")
    glicemia_input_str = input("🩸 Valor da glicemia (apenas o número em mg/dL): ")
    alimentacao_input = input("🍔 Alimentação (descrição simples, ex: 'Almoço com frango e salada'): ")
    carbos_input_str = input("🍞 Carboidratos da refeição (apenas o número em gramas, se souber, ou 0): ")
    insulina_input = input("💉 Insulina aplicada (tipo e unidades, ex: 'Rápida 8U', 'Lenta 15U', ou 'Nenhuma'): ")
    atividade_input = input("🏃 Atividade física (descrição simples, ex: 'Caminhada 30min', ou 'Nenhuma'): ")

    try:
        glicemia_input = int(glicemia_input_str)
    except ValueError:
        print("⚠️ Valor da glicemia inválido. Usando 0 para demonstração.")
        glicemia_input = 0

    try:
        carbos_input = int(carbos_input_str)
    except ValueError:
        print("⚠️ Valor dos carboidratos inválido. Usando 0 para demonstração.")
        carbos_input = 0


    print("\n--- Dados coletados. Processando com os agentes... ---")

    dados_para_monitoramento = f"{horario_input}, {glicemia_input} mg/dL, {alimentacao_input} - {carbos_input}g Carbo, {insulina_input}, {atividade_input}"


    print("\n--- Resultado do Agente de Monitoramento (Baseado nesta entrada única) ---")
    resumo_gerado = agente_monitoramento_dm1(dados_para_monitoramento)
    display(to_markdown(f"**Resumo do Registro:**\n{resumo_gerado}"))
    print("--------------------------------------------------")

    print("\n--- Resultado do Agente Conselheiro Nutricional ---")
    conselho_gerado = agente_nutricional_dm1(
        refeicao_planejada=alimentacao_input,
        glicemia_atual_contexto=glicemia_input
    )
    display(to_markdown(f"**Conselho Nutricional:**\n{conselho_gerado}"))
    print("--------------------------------------------------")

    print("\n--- Resultado do Agente de Suporte à Dosagem (EDUCACIONAL) ---")
    suporte_gerado = agente_insulina_dm1(
        glicemia_atual=glicemia_input,
        carbos_refeicao=carbos_input,
        insulina_ativa=insulina_input,
        atividade_recente=atividade_input
    )
    display(to_markdown(f"**Suporte Informativo sobre Dosagem:**\n{suporte_gerado}"))
    print("--------------------------------------------------")

    print("\n✅ Processamento com Agentes Concluído.")
    print("--------------------------------------------------")

🔬 Iniciando o Projeto Simplificado de Suporte a DM1 com Agentes 🔬
--- Por favor, insira os dados do seu registro ---
⌚ Horário do registro (HH:MM): 09:00
🩸 Valor da glicemia (apenas o número em mg/dL): 110
🍔 Alimentação (descrição simples, ex: 'Almoço com frango e salada'): Café da manhã com pão frances e café
🍞 Carboidratos da refeição (apenas o número em gramas, se souber, ou 0): 300
💉 Insulina aplicada (tipo e unidades, ex: 'Rápida 8U', 'Lenta 15U', ou 'Nenhuma'): Rápida 10u
🏃 Atividade física (descrição simples, ex: 'Caminhada 30min', ou 'Nenhuma'): nenhuma

--- Dados coletados. Processando com os agentes... ---

--- Resultado do Agente de Monitoramento (Baseado nesta entrada única) ---


> **Resumo do Registro:**
> Ok, aqui está uma análise dos dados que você forneceu para o dia 17 de maio de 2025:
> 
> Às 09:00, a glicemia foi de 110 mg/dL após a administração de 10 unidades de insulina rápida para cobrir um café da manhã com aproximadamente 300g de carboidratos. A glicemia estava dentro da faixa normal neste momento.
> 


--------------------------------------------------

--- Resultado do Agente Conselheiro Nutricional ---


> **Conselho Nutricional:**
> Olá! Um café da manhã com pão francês e café é uma opção comum, mas vamos conversar sobre como esses alimentos podem afetar sua glicemia e algumas alternativas saudáveis.
> 
> **Pão Francês:**
> 
> *   **Carboidratos:** O pão francês é rico em carboidratos, que se transformam em glicose no sangue. A quantidade de carboidratos em um pão francês pode variar, mas geralmente fica em torno de 50-60 gramas.
> *   **Impacto na Glicemia:** Por ser um carboidrato simples, o pão francês pode elevar a glicemia rapidamente.
> *   **Dicas:** Para minimizar o impacto, você pode optar por pão integral, que tem mais fibras e uma absorção mais lenta. Outra opção é combinar o pão com proteínas e gorduras saudáveis, como ovos ou abacate, para retardar a absorção dos carboidratos.
> 
> **Café:**
> 
> *   **Efeito na Glicemia:** O café puro, sem açúcar, geralmente tem pouco efeito direto na glicemia. No entanto, a cafeína pode afetar a sensibilidade à insulina em algumas pessoas, o que pode levar a um aumento nos níveis de açúcar no sangue.
> *   **Dicas:** Se você notar que o café afeta sua glicemia, experimente monitorar seus níveis após o consumo ou converse com seu médico sobre isso. Evite adicionar açúcar ou adoçantes calóricos ao café.
> 
> **Considerações:**
> 
> *   Com uma glicemia atual de 110 mg/dL, você está dentro da faixa normal. No entanto, é importante monitorar como sua glicemia responde após essa refeição.
> *   Lembre-se que estas são apenas estimativas gerais. A quantidade exata de carboidratos e o impacto na sua glicemia podem variar dependendo do tamanho da porção e da sua sensibilidade individual à insulina.
> 
> **Sugestões para um café da manhã mais equilibrado:**
> 
> 1.  **Substituições:** Trocar o pão francês por pão integral ou adicionar uma fonte de proteína, como ovos mexidos ou queijo, pode ajudar a equilibrar a refeição.
> 2.  **Fibras:** Adicionar frutas como morangos ou mirtilos pode aumentar a quantidade de fibras e nutrientes na sua refeição.
> 3.  **Gorduras Saudáveis:** Incluir abacate ou algumas nozes pode ajudar a retardar a absorção dos carboidratos e manter você saciado por mais tempo.
> 
> É sempre importante lembrar que cada pessoa com DM1 é única, e o que funciona para um pode não funcionar para outro. Consultar um nutricionista ou médico é fundamental para criar um plano alimentar individualizado que atenda às suas necessidades e objetivos!
> 


--------------------------------------------------

--- Resultado do Agente de Suporte à Dosagem (EDUCACIONAL) ---


> **Suporte Informativo sobre Dosagem:**
> Olá! É importante entender como diferentes fatores podem influenciar a sua necessidade de insulina rápida, mas **lembre-se: eu não posso calcular ou recomendar doses.** O objetivo aqui é fornecer informações educativas para que você possa ter discussões mais informadas com sua equipe de saúde.
> 
> Vamos discutir como a glicemia atual, os carboidratos na refeição, a insulina ativa e a atividade física podem influenciar a dose de insulina:
> 
> 1.  **Glicemia Atual:**
>     *   **Relevância:** A glicemia atual é um ponto de partida crucial. Se a sua glicemia está dentro da faixa desejada, pode ser que você precise de menos insulina para cobrir os carboidratos da refeição. Se estiver mais alta, pode ser necessário um pouco mais para corrigir. No seu exemplo, 110 mg/dL está dentro da faixa normal, então não seria necessária uma correção para hiperglicemia.
> 2.  **Carboidratos Planejados:**
>     *   **Relevância:** A quantidade de carboidratos que você planeja consumir é um dos principais determinantes da dose de insulina. Geralmente, existe uma proporção entre a quantidade de carboidratos e a dose de insulina necessária (por exemplo, 1 unidade de insulina para cada 10 gramas de carboidratos). 300g de carboidratos é uma quantidade muito alta, e demandaria uma dose alta de insulina também.
> 
> 3.  **Insulina Ativa (ou Insulina "a bordo"):**
>     *   **Relevância:** A insulina ativa é a quantidade de insulina da sua última dose que ainda está agindo no seu corpo. Se você aplicou 10 unidades de insulina rápida recentemente, parte dessa insulina ainda pode estar ativa e ajudando a baixar a glicemia. Levar em conta a insulina ativa pode ajudar a evitar hipoglicemia (glicemia baixa) ao aplicar uma nova dose.
> 
> É crucial lembrar que esses fatores interagem de maneira complexa e podem variar de pessoa para pessoa. **Sempre siga as orientações da sua equipe de saúde** para ajustar a dose de insulina de forma segura e eficaz.


--------------------------------------------------

✅ Processamento com Agentes Concluído.
--------------------------------------------------
